In [1]:
import os
import re
import time
import pickle
from typing import Tuple, List

from smart_open import smart_open
from gensim.utils import simple_preprocess
import spacy
import multiprocessing
#from gensim import 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse
import gensim
from gensim.corpora import Dictionary, MmCorpus
from collections import deque
from collections import namedtuple
import random
from gensim.models.coherencemodel import CoherenceModel
# explore the topics discussed in the reviews and compare how they have evolved over time
# We will define a series of functions to achieve that objective
from gensim.models import phrases
from smart_open import smart_open
from gensim.utils import simple_preprocess
import os

In [2]:
# read the reviews
pitchfork = pd.read_csv('../data/pitchfork/pitchfork.csv', low_memory=False, 
                       encoding = 'utf-8')
pitchfork.head()

,artist,album,genre,score,date,author,role,review,bnm,link,label,release_year
0,David Byrne,“…The Best Live Show of All Time” — NME EP,Rock,5.5,January 11 2019,Andy Beta,Contributor,"Viva Brother, Terris, Mansun, the Twang, Joe L...",0,https://pitchfork.com/reviews/albums/david-byr...,Nonesuch,2018.0
1,DJ Healer,Lost Lovesongs / Lostsongs Vol. 2,Electronic,6.2,January 11 2019,Chal Ravens,Contributor,"The Prince of Denmark—that is, the proper prin...",0,https://pitchfork.com/reviews/albums/dj-healer...,Planet Uterus,2019.0
2,Jorge Velez,Roman Birds,Electronic,7.9,January 10 2019,Philip Sherburne,Contributing Editor,"Jorge Velez has long been prolific, but that’s...",0,https://pitchfork.com/reviews/albums/jorge-vel...,Self-released,2019.0
3,Chandra,Transportation EPs,Rock,7.8,January 10 2019,Andy Beta,Contributor,When the Avalanches returned in 2016 after an ...,0,https://pitchfork.com/reviews/albums/chandra-t...,Telephone Explosion,2018.0
4,The Chainsmokers,Sick Boy,Electronic,3.1,January 9 2019,Larry Fitzmaurice,Contributor,We’re going to be stuck with the Chainsmokers ...,0,https://pitchfork.com/reviews/albums/the-chain...,"Disruptor,Columbia",2018.0


In [3]:
# read the stopwords
with open('../data/pitchfork/stop.txt', 'r') as f:
    stop_words = f.read().split('\n')
stop_words = stop_words+['songs', 'albums','record','records', 'album', 'sound', 'music',
                        'band', 'song', 'time', 'years','fuck', '\\xa0', 'kind','single','shit',
                         'good', '\n', 'great','\xa0', 'sing','rock','bad','guy','lyric',
                        'lot', 'sing','band','rock','man','girl','listen','day','bands','record',
                         'records', 'guitars','thing','pretty','artist','things',
                         'people','stuff','guitar']
#stop_words = set(stop_words)

In [4]:
print(len(pitchfork))
pitchfork['review'] = pitchfork['review'].astype(str)
pitchfork = pitchfork[pitchfork['review'].apply(lambda x: len(x)>200)]
pitchfork.reset_index(inplace=True)
print(len(pitchfork))

20873
20869


In [5]:
#pitchfork['review'] = pitchfork['review'].values.astype('str')
documents = pitchfork['review'].tolist()
doc_ids = pitchfork['link'].tolist()
print(len(documents),len(doc_ids))

20869 20869


In [6]:
documents = [str(doc) for doc in documents if doc]

In [7]:
def tokenize(documents: List[str], stop_words: List[str] = None, use_gpu = True, 
             model = 'en_core_web_lg') -> List[List[str]]:
    if use_gpu:
        spacy.prefer_gpu()
        print(spacy.prefer_gpu())
    # load the model
    nlp = spacy.load(model, disable=['ner', 'parser'])
    # Mark them as stop words
    for word in stop_words:
        nlp.Defaults.stop_words.add(word)
    print(nlp.Defaults.stop_words)
    docs = nlp.pipe(documents, batch_size=256,n_process=multiprocessing.cpu_count()-6)
    docs = [[token.lemma_.lower() for token in doc if not (token.is_stop or token.is_punct or token.is_digit)] for doc in docs]
    #docs = [[token.lemma_ for token in doc] for doc in docs]
    return docs

In [8]:
# create bigrams and create corpus and dictionary to feed into model
def make_bigrams(texts):
    phrases2 = phrases.Phrases(texts, min_count = 10)
    bigram = phrases.Phraser(phrases2)
    return [bigram[doc] for doc in texts]
def show_dfs_topk(docs, topk=20, dictionary = None):
        ndoc = len(docs)
        dfs_topk = sorted([(dictionary.id2token[k],fq) for k,fq in dictionary.dfs.items()], 
                          key=lambda x: x[1],reverse=True)[:topk]
        for i,(word,freq) in enumerate(dfs_topk):
            print(f'{i+1}:{word} --> {freq}/{ndoc} = {(1.0*freq/ndoc):>.13f}')
        return dfs_topk

def topk_dfs(docs, topk=20, dictionary = None):
    ndoc = len(docs)
    dfs_topk = show_dfs_topk(docs, topk=topk, dictionary = dictionary)
    return 1.0*dfs_topk[-1][-1]/ndoc

def show_dfs_topk(docs,topk=20, dictionary = None):
    ndoc = len(docs)
    dfs_topk = sorted([(dictionary.id2token[k],fq) for k,fq in dictionary.dfs.items()],
                      key=lambda x: x[1],reverse=True)[:topk]
    for i,(word,freq) in enumerate(dfs_topk):
        print(f'{i+1}:{word} --> {freq}/{ndoc} = {(1.0*freq/ndoc):>.13f}')
    return dfs_topk

In [9]:
'''
documents_tokenized = tokenize(documents[100:300], stop_words = stop_words)
documents_tokenized = make_bigrams(documents_tokenized[100:300])
#documents_tokenized = [i for i in documents_tokenized if not any(b in stop_words for b in i)]
dictionary = Dictionary(documents_tokenized)
print(f'dictionary size is {len(dictionary)}')
dictionary.id2token = {v:k for k,v in dictionary.token2id.items()} 
show_dfs_topk(documents_tokenized, topk = 20, dictionary = dictionary)
ratio = topk_dfs(documents_tokenized, topk=5, dictionary=dictionary)
print(ratio)
print(f'before compacting dict is {len(dictionary)} words')
dictionary.filter_extremes(no_below = 10, no_above = ratio)
dictionary.compactify()
print(f'after compacting dict is {len(dictionary)} words')
'''
#len(documents_tokenized)

"\ndocuments_tokenized = tokenize(documents[100:300], stop_words = stop_words)\ndocuments_tokenized = make_bigrams(documents_tokenized[100:300])\n#documents_tokenized = [i for i in documents_tokenized if not any(b in stop_words for b in i)]\ndictionary = Dictionary(documents_tokenized)\nprint(f'dictionary size is {len(dictionary)}')\ndictionary.id2token = {v:k for k,v in dictionary.token2id.items()} \nshow_dfs_topk(documents_tokenized, topk = 20, dictionary = dictionary)\nratio = topk_dfs(documents_tokenized, topk=5, dictionary=dictionary)\nprint(ratio)\nprint(f'before compacting dict is {len(dictionary)} words')\ndictionary.filter_extremes(no_below = 10, no_above = ratio)\ndictionary.compactify()\nprint(f'after compacting dict is {len(dictionary)} words')\n"

In [10]:
%%time
retokenize = False
if os.path.exists('../data/pitchfork/corpus.mm') and retokenize==False:
    print('tokenization already conducted')
else:
    print('tokenization underway')
    documents_tokenized = tokenize(documents, stop_words = stop_words)
    documents_tokenized = make_bigrams(documents_tokenized)
    #documents_tokenized = [i for i in documents_tokenized if not any(b in stop_words for b in i)]
    documents_tokenized = [[word for word in doc if word not in stop_words] for doc in documents_tokenized]
    dictionary = Dictionary(documents_tokenized)
    print(f'dictionary size is {len(dictionary)}')
    dictionary.id2token = {v:k for k,v in dictionary.token2id.items()} 
    show_dfs_topk(documents_tokenized, topk = 20, dictionary = dictionary)
    ratio = topk_dfs(documents_tokenized, topk=20, dictionary=dictionary)
    print(ratio)
    print(f'before compacting dict is {len(dictionary)} words')
    dictionary.filter_extremes(no_below = 40, no_above = ratio)
    dictionary.compactify()
    print(f'after compacting dict is {len(dictionary)} words')
    # create bows
    bows, docs = [],[]
    for doc in documents_tokenized:
        _bow = dictionary.doc2bow(doc)
        bows.append(_bow)
        docs.append(doc)
    # save the corpuss, dictionary and text
    gensim.corpora.MmCorpus.serialize('../data/pitchfork/corpus.mm', bows)
    dictionary.save_as_text('../data/pitchfork/dict.txt')
    with open('../data/pitchfork/dict.pkl','wb') as f:
        pickle.dump(dictionary,f)
    with open('../data/pitchfork/docs.pkl','wb') as f:
        pickle.dump(docs,f)
    with open('../data/pitchfork/doc_ids.pkl','wb') as f:
        pickle.dump(doc_ids,f)
    vocab_size = len(dictionary)
    num_docs = len(bows)
    print(f'Processed {len(bows)} documents.')
    # we will now create the bow representation

tokenization already conducted
CPU times: user 171 µs, sys: 53 µs, total: 224 µs
Wall time: 144 µs


In [11]:
# load dictionary
dictionary = Dictionary.load('../data/pitchfork/dict.pkl')
bows = gensim.corpora.MmCorpus('../data/pitchfork/corpus.mm')
docs = pickle.load(open('../data/pitchfork/docs.pkl','rb'))
docs_ids = pickle.load(open('../data/pitchfork/doc_ids.pkl','rb'))
print('len of vocabulary is ',len(dictionary))

len of vocabulary is  15048


In [12]:
# print
print(docs[9984], docs_ids[9984])

['closely', 'tie', 'early', 'strange', 'indie', 'hardcore_punk', 'grow', 'early', 'indie', 'art_rocker', 'plenty', 'hardcore', 'late_1980', 'early_90', 'label', 'touch', 'dischord', 'coliseum', 'house', 'curse', 'nod', 'distant_past', 'lp', 'salvation', 'recruit', 'drummer', 'find', 'label', 'move', 'metal', 'monger', 'relapse', 'post', 'rocking', 'temporary_residence', 'opportunity', 'house', 'curse', 'tribute', 'turn', 'the-90s', 'help_shape', 'invite', 'member', 'perform', 'j._robbins', 'jawbox', 'burning_airlines', 'mix', 'add', 'backing_vocal', 'blind', 'eye"--', 'hell', 'track', 'surge', 'rarely', 'hear', 'indie', 'toy', 'quiet_loud', 'dynamic', 'unpredictable', 'rhythm', 'crackle', 'distortion', 'melody', 'straightforward', 'clear', 'patterson', 'discernible', 'matter', 'sputter', 'yowl', 'nuance', 'singing', 'cloak', 'red', 'perimeter', 'subtlety', 'remarkable', 'voice', 'guttural', 'gravel', 'gargle', 'roar', 'john', 'brannon', 'negative_approach', 'laugh', 'hyenas', 'proud', 

In [13]:
# split into training and test
from sklearn.model_selection import train_test_split
x_bows_train, x_bows_test = train_test_split(bows, test_size=0.15, random_state=192)
x_tokens_train, x_tokens_test = train_test_split(docs, test_size=0.15, random_state=192)
x_ids_train, x_ids_test = train_test_split(docs_ids, test_size=0.15, random_state=192)

In [14]:
#item = list(zip(*bows[0]))
#torch.tensor(list(item[1])).float()
print(f'len of training size is {len(x_bows_train)} and tokes are {len(x_tokens_train)} and ids are {len(x_ids_train)}'
      f' len of test size is {len(x_bows_test)} and tokens are {len(x_tokens_test)} and ids are {len(x_ids_test)}')

len of training size is 17738 and tokes are 17738 and ids are 17738 len of test size is 3131 and tokens are 3131 and ids are 3131


In [15]:
from torch.utils.data import Dataset,DataLoader
import torch

class Data_Processing(object):
    def __init__(self, docs, bows, vocab, ids):
        self.docs = docs
        self.bows = bows
        self.vocab = vocab
        self.ids = ids
        
    def __getitem__(self,idx):
        bow = torch.zeros(len(self.vocab))
        # create token and frequency
        item = list(zip(*self.bows[idx])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
        # create
        bow[list(item[0])] = torch.tensor(list(item[1])).float()
        txt = self.docs[idx]
        id_ = self.ids[idx]
        #print(f'shape of bow before being put together in data loader {bow.shape} {type(bow)}')
        return txt, bow, id_
    
    def __len__(self):
        return len(self.docs)
    
    def collate_fn1(self,batch_data):
        texts,bows,id_ = list(zip(*batch_data))
        #print(f'what happens with collate function {torch.stack(bows,dim=0)}, {torch.stack(bows,dim=0).shape}')
        return texts,torch.stack(bows,dim=0),id_

    def __iter__(self):
        for doc in self.docs:
            yield doc

batch_size = 512

# create a class to process the traininga and test data
training_data = Data_Processing(x_tokens_train, x_bows_train, dictionary, x_ids_train)
test_data = Data_Processing(x_tokens_test, x_bows_test, dictionary, x_ids_test)

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn1),
                    'test': DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=test_data.collate_fn1)}
dataset_sizes = {'train':len(training_data)}
example = next(iter(dataloaders_dict.get('train')))

In [16]:
print(example[2][0],example[0][0])

https://pitchfork.com/reviews/albums/13207-red/ ['countless', 'hedge', 'heap', 'irony', 'arguably', 'vice_versa', 'case', 'buttress', "datarock'sâ", '\xa0 ', 'technical_prowess', 'norwegian_duo', 'eye_roll', 'moment', 'night_flight', 'uranus', 'dance', 'daddy', 'sport', 'slick', 'dance', 'production', 'biz', 'mention', 'effortless', 'traverse', 'neurotic', 'post_punk', 'lick', 'coke', 'gill', '1980s_synth', 'pop', 'convention', 'sophomore_lp', 'red', 'share', 'grow', 'moment', 'artistically', 'mature', 'high_concept', 'jury_rig', 'genre', 'kitsch', 'disco', 'legion', 'talk_heads', 'eno', 'knock_off', 'float', 'today', 'red', 'austere', 'lapse', 'wistfulness', 'jolt', 'fredrik', 'saroea', 'ketil', 'mosnes', 'bizarre', 'stable', 'niche', 'carve', 'debut', 'quick_glance', 'track_listing', 'show', 'datarock', 'pop', 'red', 'boil', 'compact', 'minute', 'nut', 'dramatic', 'dancefloor', 'build', 'premise', 'shout', 'goofy', 'shibboleth', 'consist', 'talking_heads', 'title', 'true', 'stories',

In [17]:
# define the model
import torch
import torch.nn.functional as F
from torch import nn


class ETM(nn.Module):
    # instantiate the model
    def __init__(self, device = None, num_topics = 20, vocab_size = None, t_hidden_size = 1024, 
                 rho_size = 300, emb_size=None, theta_act='relu',
                 embeddings=None, train_embeddings=True, enc_drop=0.5):
        '''
        Creates an embedded topic model instance. The model hyperparameters are:
        device: if using cuda or cpu
        num_topics: number of topics to be used
        vocab_size: size of the vocabulary
        t_hidden_size: size of the hidden embedding   
        embeddings (str or KeyedVectors): KeyedVectors instance containing word-vector mapping for embeddings, 
                                          or its path. None by default
        rho_size (int): dimension of rho (embeddings vector)
        emb_size (int): dimens-ion of embeddings if not trained jointly, default to None
        theta_act (str): activation function can be tanh, softplus, relu, rrelu, leakyrelu, elu, selu, glu, etc.
        train_embeddings (bool): whether to fix rho or train it, defaults to True
        enc_drop: dropout value for activation function, defaults to 0.5
        '''
        super(ETM, self).__init__()
        # define hyperparameters
        self.num_topics = num_topics
        self.vocab_size = vocab_size
        # in case we want to train the embeddings this parameter controls the hidden size of the embeddings
        self.rho_size = rho_size
        # dimension of the matrix for the topic mixture (topics hidden size)
        self.t_hidden_size = t_hidden_size
        # dropout to be applied to the encoder
        self.enc_drop = enc_drop
        # size of the embeddings in case a pretrained embedding is provided
        self.emb_size = emb_size
        # dropout rate
        self.t_drop = nn.Dropout(enc_drop)
        #
        self.train_embeddings = train_embeddings
        
        # activation for theta
        self.theta_act = self.get_activation(theta_act)
        # device to use for training
        self.device = device

        # define the word embedding matrix \rho
        if self.train_embeddings:
            self.rho = nn.Linear(rho_size, vocab_size, bias=True)
        else:
            # get the dimension of the embeddings
            print(embeddings.size())
            num_embeddings, emb_size = embeddings.size()
            # instantiate rho embeddings
            self.rho = embeddings.clone().float().to(self.device)

        # define the matrix containing the topic embeddings
        self.alphas = nn.Linear(rho_size, num_topics, bias=False)

        # define variational distribution for \theta_{1:D} via amortization
        # this takes the vocabulary size, creates the hidden size for the
        # topics matrix, applies activation function, applies another linear
        # layer followed by additional activation function
        self.q_theta = nn.Sequential(nn.Linear(vocab_size, t_hidden_size),
            self.theta_act,
            nn.Linear(t_hidden_size, t_hidden_size),
            self.theta_act,
        )
        
        # linear transformation of the topics hidden size to the number of topics
        self.mu_q_theta = nn.Linear(t_hidden_size, num_topics, bias=True)
        
        # another linear transformation from hidden size to number of topics
        self.logsigma_q_theta = nn.Linear(t_hidden_size, num_topics, bias=True)

    def get_activation(self, act = 'relu'):
        '''
        Define the activation function options available are tahn, relu, softplus, rrelu, 
        leakyrelu, elu, selu and glu.
        '''
        if act == 'tanh':
            act = nn.Tanh()
        elif act == 'relu':
            act = nn.ReLU()
        elif act == 'softplus':
            act = nn.Softplus()
        elif act == 'rrelu':
            act = nn.RReLU()
        elif act == 'leakyrelu':
            act = nn.LeakyReLU()
        elif act == 'elu':
            act = nn.ELU()
        elif act == 'selu':
            act = nn.SELU()
        elif act == 'glu':
            act = nn.GLU()
        else:
            act = nn.Tanh()
        return act

    def reparameterize(self, mu, logvar):
        """
        Returns a sample from a Gaussian distribution via reparameterization.
        mu stands for
        logvar stands for 
        reference for reparametrizetion trick. 
        https://stats.stackexchange.com/questions/199605/how-does-the-reparameterization-trick-for-vaes-work-and-why-is-it-important
        """
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul_(std).add_(mu)
        else:
            return mu

    def encode(self, bows):
        """Returns paramters of the variational distribution for \theta.
        input: bows batch of bag-of-words...tensor of shape bsz x V (batch size* vocabulary)
        output: mu_theta, log_sigma_theta
        """
        
        q_theta = self.q_theta(bows)
        if self.enc_drop > 0:
            q_theta = self.t_drop(q_theta)
        # mu_q_theta takes the topic hidden size and outputs a vector with the dimension of the topics
        mu_theta = self.mu_q_theta(q_theta)
        # takes the topic hidden size and outpus another vector equal to the size of the vectors
        logsigma_theta = self.logsigma_q_theta(q_theta)
        # returns kl theta which is a normalized number
        kl_theta = -0.5 * torch.sum(1 + logsigma_theta - mu_theta.pow(2) - logsigma_theta.exp(), dim=-1).mean()
        #print(f'this is kl_theta {kl_theta}')
        return mu_theta, logsigma_theta, kl_theta
       
    def get_beta(self):
        """
        This generate the description as a defintion over words
        Returns:
            [type]: [description]
        """
        try:
            logit = self.alphas(self.rho.weight) # torch.mm(self.rho, self.alphas)
        except:
            logit = self.alphas(self.rho)
        # gets logits over vocabulary dimension when fitting to the topic
        beta = F.softmax(logit, dim=0).transpose(1, 0) ## softmax over vocab dimension
        #print(f'this is beta {beta}')
        return beta

    def get_theta(self, normalized_bows):
        """
        getting the topic poportion for the document passed in the normalixe bow or tf-idf
        """
        # gets mu_theta (linear layer for topic dimension), logsima_theta another liner layer output
        # and the kld_tjet
        mu_theta, logsigma_theta, kld_theta = self.encode(normalized_bows)
        # gets variable to backpropagate, returns  reparametized mu theta
        z = self.reparameterize(mu_theta, logsigma_theta)
        # applied softmax and returns squeezed logits
        theta = F.softmax(z, dim=-1) 
        #print(f'this is theta {theta}')
        return theta, kld_theta

    def decode(self, theta, beta):
        """compute the probability of topic given the document which is equal to theta^T ** B
        Args:
            theta ([type]): vector of proabilities of belonging to a given topic
            beta ([type]): probability of belonging to a given topic
        Returns:
            [type]: [description]
        """
        # multiplication of softmax with beta (likelihood of belonging to a topic given likelihood a word
        # belongs to another topic)
        res = torch.mm(theta, beta)
        # normalizes predictions
        almost_zeros = torch.full_like(res, 1e-6)
        results_without_zeros = res.add(almost_zeros)
        predictions = torch.log(results_without_zeros)
        return predictions

    def forward(self, bows, normalized_bows, theta=None, aggregate=True):
        # get \theta
        if theta is None:
            theta, kld_theta = self.get_theta(normalized_bows)
        else:
            kld_theta = None

        # get \beta
        beta = self.get_beta()

        # get prediction loss
        preds = self.decode(theta, beta)
        recon_loss = -(preds * bows).sum(1)
        if aggregate:
            recon_loss = recon_loss.mean()
        return recon_loss, kld_theta

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [19]:
model = ETM(device = device, num_topics = 20, vocab_size =len(dictionary), 
            t_hidden_size = 1024, rho_size = 400, 
            emb_size=None, theta_act='tahn',
            embeddings=None, train_embeddings=True, enc_drop=0.5)

In [20]:
model

ETM(
  (t_drop): Dropout(p=0.5, inplace=False)
  (theta_act): Tanh()
  (rho): Linear(in_features=400, out_features=15048, bias=True)
  (alphas): Linear(in_features=400, out_features=20, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=15048, out_features=1024, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): Tanh()
  )
  (mu_q_theta): Linear(in_features=1024, out_features=20, bias=True)
  (logsigma_q_theta): Linear(in_features=1024, out_features=20, bias=True)
)

In [21]:
from torch.optim import lr_scheduler, AdamW, SGD,Adam
lr = 0.003
optimizer = Adam(model.parameters(), lr=lr, weight_decay = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.003
    weight_decay: 0.001
)

In [22]:
#model.forward()
def get_topics(model = None, num_topics = 25, top_n_words=10, vocabulary = None) -> List[str]:
        """
        Gets topics. By default, returns the 10 most relevant terms for each topic.
        Parameters:
        ===
            top_n_words (int): number of top words per topic to return
        Returns:
        ===
            list of str: topic list
        """

        with torch.no_grad():
            topics = []
            gammas = model.get_beta()

            for k in range(model.num_topics):
                gamma = gammas[k]
                top_words = list(gamma.cpu().numpy().argsort()
                                 [-top_n_words:][::-1])
                topic_words = [vocabulary[a] for a in top_words]
                topics.append(topic_words)

            return topics

def get_topic_words(model,dictionary):
    with torch.no_grad():
        topic_words = []
        gammas = model.get_beta()
        
        for k in range(model.num_topics):
            gamma = gammas[k]
            top_words = list(gamma.cpu().numpy().argsort()[::-1])
            tp = [dictionary[a] for a in top_words]
            topic_words.append(tp)
    
    return topic_words
        
def calc_topic_diversity(topic_words):
    '''topic_words is in the form of [[w11,w12,...],[w21,w22,...]]'''
    vocab = set(sum(topic_words,[]))
    n_total = len(topic_words) * len(topic_words[0])
    print(len(topic_words))
    topic_div = len(vocab) / n_total
    return topic_div


def get_topic_diversity(model, topk=50):
    with torch.no_grad():
        beta = model.get_beta()
        num_topics = model.num_topics
        print(beta.shape, num_topics)
        list_w = np.zeros((num_topics, topk))
        print(list_w.shape)
        for k in range(num_topics):
            idx = beta[k,:].cpu().numpy().argsort()[-topk:][::-1]
            list_w[k,:] = idx
        n_unique = len(np.unique(list_w))
        TD = n_unique / (topk * num_topics)
        #print('Topic diveristy is: {}'.format(TD))
        return TD
    
def coherence_data(topics = None, texts = None, dictionary = None):
    c_uci_coherence_model = CoherenceModel(topics=topics,texts=docs,dictionary=dictionary,coherence='c_uci')
    c_uci_score = c_uci_coherence_model.get_coherence()
    c_npmi_coherence_model = CoherenceModel(topics=topics,texts=docs,dictionary=dictionary,coherence='c_npmi')
    c_npmi_score = c_npmi_coherence_model.get_coherence()
    return c_uci_score,c_npmi_score


In [54]:
from pprint import pprint
def nearest_neighbors(word, embeddings, vocab, n_most_similar=5):
    vectors = embeddings.data.cpu().numpy()
    index = dictionary.token2id[word]
    query = vectors[index]
    ranks = vectors.dot(query).squeeze()
    denom = query.T.dot(query).squeeze()
    denom = denom * np.sum(vectors**2, 1)
    denom = np.sqrt(denom)
    ranks = ranks / denom
    mostSimilar = []
    [mostSimilar.append(idx) for idx in ranks.argsort()[::-1]]
    nearest_neighbors = mostSimilar[1:n_most_similar]
    nearest_neighbors = [vocab[comp] for comp in nearest_neighbors]
    return nearest_neighbors

def get_most_similar_words(model = None, queries=[], vocabulary = None, n_most_similar=5) -> dict:
        """
        Gets the nearest neighborhoring words for a list of tokens. By default, returns the 20 most similar words for each token in 'queries' array.
        Parameters:
        ===
            queries (list of str): words to find similar ones
            n_most_similar (int): number of most similar words to get for each word given in the input. By default is 20
        Returns:
        ===
            dict of (str, list of str): dictionary containing the mapping between query words given and their respective similar words
        """

        model.eval()

        # visualize word embeddings by using V to get nearest neighbors
        with torch.no_grad():
            embeddings = model.rho.weight  # Vocab_size x E
            

            neighbors = {}
            for word in queries:
                neighbors[word] = nearest_neighbors(
                    word,embeddings, vocabulary, n_most_similar)

            return neighbors
        


In [24]:
import wandb
wandb.init(entity="jlealtru", project="ETM_runs_p")
config = wandb.config          # Initialize config

def train(model, batch_size=256,dictionary = None,
          learning_rate=2e-3,test_data=None,
          num_epochs=600,is_evaluate=False,log_every=40,ckpt=None):
    model.to(device)
    model.train()
    
    data_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn1)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    
    
    if ckpt:
        self.load_model(ckpt["net"])
        optimizer.load_state_dict(ckpt["optimizer"])
        start_epoch = ckpt["epoch"] + 1
    else:
        start_epoch = 0
    
    acc_loss = 0
    acc_kl_theta_loss = 0
    cnt = 0
    
    trainloss_lst, valloss_lst = [], []
    recloss_lst, klloss_lst = [],[]
    c_v_lst, c_w2v_lst, c_uci_lst, c_npmi_lst, mimno_tc_lst, td_lst = [], [], [], [], [], []
    for epoch in range(start_epoch, num_epochs):
        epochloss_lst = []
        model.train()
        for iter_,data in enumerate(data_loader):
            #optimizer.zero_grad()
            model.zero_grad(set_to_none=True)
            
            txts,bows,ids = data
            bows = bows.to(device)
            normalized_bows = bows
            normalized_bows.to(device)
            
            recon_loss, kld_theta = model.forward(bows, normalized_bows)
            total_loss = recon_loss + kld_theta
            total_loss.backward()
            optimizer.step()

            acc_loss += torch.sum(recon_loss).item()
            acc_kl_theta_loss += torch.sum(kld_theta).item()
            cnt += 1
            if iter_ % 4 == 0:
                cur_loss = round(acc_loss / cnt, 2) 
                cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
                cur_real_loss = round(cur_loss + cur_kl_theta, 2)

                print('Epoch: {} KL_theta: is {} .. Rec_loss: {} .. NELBO: {}'.format(
                    epoch, cur_kl_theta, cur_loss, cur_real_loss))
                
                # add wandb
                wandb.log({"Epoch": epoch,
                           "Train Loss": cur_kl_theta,"rec_loss": cur_loss,'NELBO':cur_real_loss,
                          "lr": learning_rate,
                          "optimizer": 'Adam'})

        cur_loss = round(acc_loss / cnt, 2) 
        cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
        cur_real_loss = round(cur_loss + cur_kl_theta, 2)
        print('*'*100)
        print('Epoch: {} KL_theta: is {} .. Rec_loss: {} .. NELBO: {}'.format(
                    epoch, cur_kl_theta, cur_loss, cur_real_loss))
        if (epoch+1)%log_every==0:
            topic_words = get_topic_words(model, dictionary)
            topic_diversity = get_topic_diversity(model,topk=200)
            coh_scores = coherence_data(topics = topic_words, texts = txts, dictionary = dictionary)
            #calc_topic_diversity(topic_words)
            print(f'topic diversity is {topic_diversity}')
            pprint(get_topics(model = model, num_topics = model.num_topics, top_n_words= 10, vocabulary = dictionary))
            wandb.log({"Epoch": epoch,
                       "Train Loss": cur_kl_theta,"rec_loss": cur_loss,'NELBO':cur_real_loss,
                       "lr": learning_rate,
                       "optimizer": 'Adam',
                      'topic_diversity': topic_diversity,
                       'uci': coh_scores[0],
                       'npmi':coh_scores[1]
                      })

wandb: Currently logged in as: jlealtru (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [25]:
#model.to(device)
wandb.watch(model, log="all")
train(model,batch_size=1024, learning_rate=2e-3,test_data=None,dictionary=dictionary,
      num_epochs=800,is_evaluate=False,log_every=40,
      ckpt=None)

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch: 0 KL_theta: is 0.03 .. Rec_loss: 2119.38 .. NELBO: 2119.41
Epoch: 0 KL_theta: is 8.23 .. Rec_loss: 2146.25 .. NELBO: 2154.48
Epoch: 0 KL_theta: is 5.78 .. Rec_loss: 2152.41 .. NELBO: 2158.19
Epoch: 0 KL_theta: is 4.77 .. Rec_loss: 2144.07 .. NELBO: 2148.84
Epoch: 0 KL_theta: is 3.98 .. Rec_loss: 2126.31 .. NELBO: 2130.29
****************************************************************************************************
Epoch: 0 KL_theta: is 3.83 .. Rec_loss: 2124.76 .. NELBO: 2128.59
Epoch: 1 KL_theta: is 3.67 .. Rec_loss: 2123.0 .. NELBO: 2126.67
Epoch: 1 KL_theta: is 3.22 .. Rec_loss: 2106.19 .. NELBO: 2109.41
Epoch: 1 KL_theta: is 2.88 .. Rec_loss: 2092.28 .. NELBO: 2095.16
Epoch: 1 KL_theta: is 2.58 .. Rec_loss: 2081.86 .. NELBO: 2084.44
Epoch: 1 KL_theta: is 2.32 .. Rec_loss: 2066.27 .. NELBO: 2068.59
****************************************************************************************************
Epoch: 1 KL_theta: is 2.27 .. Rec_loss: 2062.11 .. NELBO: 2064.38
Epoch: 

****************************************************************************************************
Epoch: 16 KL_theta: is 0.8 .. Rec_loss: 1963.16 .. NELBO: 1963.96
Epoch: 17 KL_theta: is 0.8 .. Rec_loss: 1963.04 .. NELBO: 1963.84
Epoch: 17 KL_theta: is 0.82 .. Rec_loss: 1962.67 .. NELBO: 1963.49
Epoch: 17 KL_theta: is 0.84 .. Rec_loss: 1962.67 .. NELBO: 1963.51
Epoch: 17 KL_theta: is 0.86 .. Rec_loss: 1962.51 .. NELBO: 1963.37
Epoch: 17 KL_theta: is 0.87 .. Rec_loss: 1962.11 .. NELBO: 1962.98
****************************************************************************************************
Epoch: 17 KL_theta: is 0.88 .. Rec_loss: 1962.23 .. NELBO: 1963.11
Epoch: 18 KL_theta: is 0.89 .. Rec_loss: 1962.2 .. NELBO: 1963.09
Epoch: 18 KL_theta: is 0.91 .. Rec_loss: 1962.29 .. NELBO: 1963.2
Epoch: 18 KL_theta: is 0.93 .. Rec_loss: 1961.98 .. NELBO: 1962.91
Epoch: 18 KL_theta: is 0.95 .. Rec_loss: 1961.67 .. NELBO: 1962.62
Epoch: 18 KL_theta: is 0.96 .. Rec_loss: 1961.27 .. NELBO: 1962.2

Epoch: 33 KL_theta: is 2.07 .. Rec_loss: 1951.05 .. NELBO: 1953.12
Epoch: 33 KL_theta: is 2.09 .. Rec_loss: 1950.89 .. NELBO: 1952.98
Epoch: 33 KL_theta: is 2.1 .. Rec_loss: 1950.76 .. NELBO: 1952.86
Epoch: 33 KL_theta: is 2.12 .. Rec_loss: 1950.65 .. NELBO: 1952.77
****************************************************************************************************
Epoch: 33 KL_theta: is 2.12 .. Rec_loss: 1950.65 .. NELBO: 1952.77
Epoch: 34 KL_theta: is 2.13 .. Rec_loss: 1950.59 .. NELBO: 1952.72
Epoch: 34 KL_theta: is 2.14 .. Rec_loss: 1950.57 .. NELBO: 1952.71
Epoch: 34 KL_theta: is 2.16 .. Rec_loss: 1950.29 .. NELBO: 1952.45
Epoch: 34 KL_theta: is 2.18 .. Rec_loss: 1950.19 .. NELBO: 1952.37
Epoch: 34 KL_theta: is 2.19 .. Rec_loss: 1950.17 .. NELBO: 1952.36
****************************************************************************************************
Epoch: 34 KL_theta: is 2.2 .. Rec_loss: 1950.06 .. NELBO: 1952.26
Epoch: 35 KL_theta: is 2.2 .. Rec_loss: 1950.08 .. NELBO: 1952.

Epoch: 45 KL_theta: is 2.9 .. Rec_loss: 1945.42 .. NELBO: 1948.32
Epoch: 45 KL_theta: is 2.92 .. Rec_loss: 1945.33 .. NELBO: 1948.25
Epoch: 45 KL_theta: is 2.93 .. Rec_loss: 1945.28 .. NELBO: 1948.21
Epoch: 45 KL_theta: is 2.95 .. Rec_loss: 1945.15 .. NELBO: 1948.1
****************************************************************************************************
Epoch: 45 KL_theta: is 2.95 .. Rec_loss: 1945.13 .. NELBO: 1948.08
Epoch: 46 KL_theta: is 2.95 .. Rec_loss: 1945.13 .. NELBO: 1948.08
Epoch: 46 KL_theta: is 2.97 .. Rec_loss: 1945.04 .. NELBO: 1948.01
Epoch: 46 KL_theta: is 2.99 .. Rec_loss: 1944.91 .. NELBO: 1947.9
Epoch: 46 KL_theta: is 3.0 .. Rec_loss: 1944.79 .. NELBO: 1947.79
Epoch: 46 KL_theta: is 3.02 .. Rec_loss: 1944.73 .. NELBO: 1947.75
****************************************************************************************************
Epoch: 46 KL_theta: is 3.02 .. Rec_loss: 1944.76 .. NELBO: 1947.78
Epoch: 47 KL_theta: is 3.02 .. Rec_loss: 1944.74 .. NELBO: 1947.7

...

Epoch: 739 KL_theta: is 11.62 .. Rec_loss: 1904.93 .. NELBO: 1916.55
Epoch: 739 KL_theta: is 11.62 .. Rec_loss: 1904.93 .. NELBO: 1916.55
****************************************************************************************************
Epoch: 739 KL_theta: is 11.62 .. Rec_loss: 1904.93 .. NELBO: 1916.55
Epoch: 740 KL_theta: is 11.62 .. Rec_loss: 1904.93 .. NELBO: 1916.55
Epoch: 740 KL_theta: is 11.62 .. Rec_loss: 1904.93 .. NELBO: 1916.55
Epoch: 740 KL_theta: is 11.62 .. Rec_loss: 1904.93 .. NELBO: 1916.55
Epoch: 740 KL_theta: is 11.63 .. Rec_loss: 1904.93 .. NELBO: 1916.56
Epoch: 740 KL_theta: is 11.63 .. Rec_loss: 1904.93 .. NELBO: 1916.56
****************************************************************************************************
Epoch: 740 KL_theta: is 11.63 .. Rec_loss: 1904.92 .. NELBO: 1916.55
Epoch: 741 KL_theta: is 11.63 .. Rec_loss: 1904.92 .. NELBO: 1916.55
Epoch: 741 KL_theta: is 11.63 .. Rec_loss: 1904.91 .. NELBO: 1916.54
Epoch: 741 KL_theta: is 11.63 .. Rec_lo

Epoch: 755 KL_theta: is 11.65 .. Rec_loss: 1904.83 .. NELBO: 1916.48
Epoch: 755 KL_theta: is 11.65 .. Rec_loss: 1904.82 .. NELBO: 1916.47
****************************************************************************************************
Epoch: 755 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 756 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 756 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 756 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 756 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 756 KL_theta: is 11.65 .. Rec_loss: 1904.8 .. NELBO: 1916.45
****************************************************************************************************
Epoch: 756 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 757 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 757 KL_theta: is 11.65 .. Rec_loss: 1904.81 .. NELBO: 1916.46
Epoch: 757 KL_theta: is 11.65 .. Rec_los

Epoch: 767 KL_theta: is 11.67 .. Rec_loss: 1904.74 .. NELBO: 1916.41
Epoch: 767 KL_theta: is 11.67 .. Rec_loss: 1904.74 .. NELBO: 1916.41
Epoch: 767 KL_theta: is 11.67 .. Rec_loss: 1904.74 .. NELBO: 1916.41
Epoch: 767 KL_theta: is 11.67 .. Rec_loss: 1904.73 .. NELBO: 1916.4
Epoch: 767 KL_theta: is 11.67 .. Rec_loss: 1904.73 .. NELBO: 1916.4
****************************************************************************************************
Epoch: 767 KL_theta: is 11.67 .. Rec_loss: 1904.73 .. NELBO: 1916.4
Epoch: 768 KL_theta: is 11.67 .. Rec_loss: 1904.73 .. NELBO: 1916.4
Epoch: 768 KL_theta: is 11.67 .. Rec_loss: 1904.73 .. NELBO: 1916.4
Epoch: 768 KL_theta: is 11.67 .. Rec_loss: 1904.73 .. NELBO: 1916.4
Epoch: 768 KL_theta: is 11.67 .. Rec_loss: 1904.73 .. NELBO: 1916.4
Epoch: 768 KL_theta: is 11.67 .. Rec_loss: 1904.72 .. NELBO: 1916.39
****************************************************************************************************
Epoch: 768 KL_theta: is 11.67 .. Rec_loss: 190

Epoch: 783 KL_theta: is 11.69 .. Rec_loss: 1904.63 .. NELBO: 1916.32
Epoch: 783 KL_theta: is 11.69 .. Rec_loss: 1904.62 .. NELBO: 1916.31
Epoch: 783 KL_theta: is 11.69 .. Rec_loss: 1904.63 .. NELBO: 1916.32
Epoch: 783 KL_theta: is 11.69 .. Rec_loss: 1904.63 .. NELBO: 1916.32
Epoch: 783 KL_theta: is 11.69 .. Rec_loss: 1904.62 .. NELBO: 1916.31
****************************************************************************************************
Epoch: 783 KL_theta: is 11.69 .. Rec_loss: 1904.62 .. NELBO: 1916.31
Epoch: 784 KL_theta: is 11.69 .. Rec_loss: 1904.62 .. NELBO: 1916.31
Epoch: 784 KL_theta: is 11.69 .. Rec_loss: 1904.62 .. NELBO: 1916.31
Epoch: 784 KL_theta: is 11.69 .. Rec_loss: 1904.61 .. NELBO: 1916.3
Epoch: 784 KL_theta: is 11.69 .. Rec_loss: 1904.61 .. NELBO: 1916.3
Epoch: 784 KL_theta: is 11.69 .. Rec_loss: 1904.61 .. NELBO: 1916.3
****************************************************************************************************
Epoch: 784 KL_theta: is 11.69 .. Rec_loss:

Epoch: 799 KL_theta: is 11.71 .. Rec_loss: 1904.52 .. NELBO: 1916.23
Epoch: 799 KL_theta: is 11.71 .. Rec_loss: 1904.52 .. NELBO: 1916.23
Epoch: 799 KL_theta: is 11.71 .. Rec_loss: 1904.51 .. NELBO: 1916.22
Epoch: 799 KL_theta: is 11.71 .. Rec_loss: 1904.51 .. NELBO: 1916.22
****************************************************************************************************
Epoch: 799 KL_theta: is 11.71 .. Rec_loss: 1904.51 .. NELBO: 1916.22
torch.Size([20, 15048]) 20
(20, 200)
topic diversity is 0.365
[['melody',
  'drum',
  'piano',
  'string',
  'instrumental',
  'percussion',
  'acoustic',
  'open',
  'post',
  'build'],
 ['metal',
  'riff',
  'black_metal',
  'noise',
  'hardcore',
  'heavy',
  'doom',
  'drum',
  'death',
  'black'],
 ['smith',
  'pollard',
  'bowie',
  'solo',
  'set',
  'indie',
  'cover',
  'classic',
  'fall',
  'punk'],
 ['world',
  'black',
  'life',
  'political',
  'woman',
  'write',
  'power',
  'american',
  'war',
  'america'],
 ['kid',
  'joke',
  'b

In [33]:
# get topic proportion
#with model.eval():
#    model.get_theta(x_bows_train[0])
get_most_similar_words(model = model, queries=['wilco'], vocabulary = dictionary, n_most_similar=5)

{'wilco': ['adams', 'appalachian_folk', 'oldham', 'bluegrass']}

In [100]:
# get topic words
get_topics(model = model, num_topics = model.num_topics, top_n_words= 20, vocabulary = dictionary)

[['melody',
  'drum',
  'piano',
  'string',
  'instrumental',
  'percussion',
  'acoustic',
  'open',
  'post',
  'build',
  'organ',
  'folk',
  'bass',
  'slow',
  'instrument',
  'note',
  'begin',
  'bit',
  'light',
  'quiet'],
 ['metal',
  'riff',
  'black_metal',
  'noise',
  'hardcore',
  'heavy',
  'doom',
  'drum',
  'death',
  'black',
  'drummer',
  'scream',
  'suggest',
  'group',
  'bass',
  'guitarist',
  'slow',
  'lead',
  'start',
  'year'],
 ['smith',
  'pollard',
  'bowie',
  'solo',
  'set',
  'indie',
  'cover',
  'classic',
  'fall',
  'punk',
  'year',
  'springsteen',
  'live',
  'line',
  'early',
  'nirvana',
  'title',
  'suggest',
  'prove',
  'sort'],
 ['world',
  'black',
  'life',
  'political',
  'woman',
  'write',
  'power',
  'american',
  'war',
  'america',
  'word',
  'year',
  'prince',
  'culture',
  'punk',
  'call',
  'art',
  'white',
  'politic',
  'message'],
 ['kid',
  'joke',
  'boy',
  'fun',
  'party',
  'funny',
  'call',
  'sex',
  

In [55]:
tuple_ = document_inference(model = model, dataloader = dataloaders_dict.get('train'), dictionary = dictionary)


/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


21


In [76]:
def document_inference(model = None, dataloader = None, dictionary = None):
    topic_list = []
    document_id = []
    with torch.no_grad():
        for iter_,data in enumerate(dataloader):
            txts,bows,ids = data
            # create the bow representation of the document for inference with zeros
            #bow = torch.zeros(len(dictionary))
            # get the counts and tokens
            #item = list(zip(*bows)) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
            # replace 0 withs actual counts
            #bow[list(item[0])] = torch.tensor(list(item[1])).float()
            # get the topic tensor
            bows = bows.to(device)
            topics,_ = model.get_theta(bows)
            t = topics.clone().detach().cpu().numpy()
            topic_list.append(t)
            document_id.append(ids)
    # create data frame
    column_name = ['document_id']
    for i in range(model.num_topics):
        column = 'topic '+str(i)
        column_name.append(column)
    print(len(column_name))
    #topic_dataframe = pd.DataFrame(zip(document_id,topic_list),
    #                               columns = column_name)
    return topic_list,document_id,column_name

In [77]:
a = document_inference(model = model, dataloader = dataloaders_dict.get('test'), dictionary=dictionary)

21


In [103]:
pd.DataFrame(a[0][0])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.368662,0.102460,0.028979,0.017429,0.020942,0.013696,0.092804,0.087597,0.015644,0.013093,0.009968,0.021574,0.023503,0.032703,0.012570,0.031591,0.035810,0.025536,0.025042,0.020398
1,0.051981,0.024703,0.040526,0.039057,0.154940,0.312661,0.009459,0.017642,0.010536,0.023985,0.036981,0.032960,0.044112,0.019189,0.017618,0.034224,0.039332,0.027864,0.046254,0.015977
2,0.075027,0.020519,0.015498,0.013055,0.259952,0.009390,0.017711,0.055814,0.017012,0.018520,0.020351,0.178827,0.012561,0.138891,0.021513,0.010553,0.028264,0.018466,0.030004,0.038071
3,0.260663,0.016734,0.033439,0.010657,0.012177,0.008602,0.012741,0.106373,0.014764,0.022798,0.013887,0.170228,0.020911,0.126499,0.010513,0.016290,0.081931,0.013618,0.009736,0.037439
4,0.099167,0.024256,0.042715,0.052832,0.178057,0.025691,0.052712,0.132950,0.022579,0.092698,0.036922,0.070261,0.019600,0.021834,0.015948,0.011464,0.006508,0.038570,0.038741,0.016495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.008883,0.007674,0.028372,0.007316,0.008477,0.005740,0.012067,0.609275,0.008888,0.010725,0.011952,0.018011,0.012607,0.024617,0.009927,0.014166,0.156976,0.012302,0.014964,0.017059
508,0.015384,0.022799,0.019916,0.020233,0.076952,0.016571,0.034105,0.111422,0.014974,0.011967,0.041721,0.012156,0.028188,0.336845,0.012543,0.037976,0.045689,0.040136,0.081946,0.018478
509,0.011341,0.088401,0.033344,0.154978,0.113634,0.038601,0.250270,0.010032,0.018125,0.028039,0.012844,0.012733,0.071305,0.008379,0.009995,0.028898,0.020519,0.060171,0.019366,0.009024
510,0.235376,0.018591,0.012360,0.011228,0.008675,0.027193,0.050714,0.035624,0.013283,0.023831,0.017597,0.021026,0.022251,0.074060,0.022675,0.013087,0.035052,0.024729,0.282817,0.049828


In [ ]:
with torch.no_grad():
    bow = torch.zeros(len(dictionary))
    item = list(zip(*x_bows_train[1])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
    bow[list(item[0])] = torch.tensor(list(item[1])).float()
    topics = model.get_theta(torch.tensor(bow).float().to(device))
    print(f'topic is: {torch.argmax(topics[0])} and text is {x_tokens_train[1]}')

In [ ]:
topics

In [46]:
# save model
torch.save(model.state_dict(), '../data/pitchfork/etm_original_architecture_400d.pth')

In [ ]:
get_most_similar_words(model = model, queries=['pop','cash'], vocabulary = vocab, n_most_similar=5)